<a href="https://colab.research.google.com/github/OdysseusPolymetis/colabs_for_nlp/blob/main/4_vecteurs_et_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim matplotlib scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE

In [ ]:
# 1. Chargement d'un modèle d'embedding pré-entraîné (version légère)
# Téléchargement d'un modèle Word2Vec pré-entraîné en français
!wget -O frwiki.gensim https://zenodo.org/records/6004718/files/roman20_mdt=skgr-opt=negsam-itr=10-dim=300-win=6-min=50.gensim?download=1

--2025-04-03 09:33:40--  https://zenodo.org/records/6004718/files/roman20_mdt=skgr-opt=negsam-itr=10-dim=300-win=6-min=50.gensim?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107458244 (102M) [application/octet-stream]
Saving to: ‘frwiki.gensim’

frwiki.gensim       100%[===================>] 102.48M  3.83MB/s    in 1m 59s  

2025-04-03 09:35:40 (883 KB/s) - ‘frwiki.gensim’ saved [107458244/107458244]



In [ ]:
model = KeyedVectors.load('frwiki.gensim')

In [ ]:
literary_terms = ["roman_nom", "poésie_nom", "auteur_nom", "personnage_nom", "métaphore_nom"]
for term in literary_terms:
    try:
        similar_words = model.wv.most_similar(term, topn=5)
        print(f"\nMots similaires à '{term}':")
        for word, similarity in similar_words:
            print(f"  {word}: {similarity:.4f}")
    except KeyError:
        print(f"Le mot '{term}' n'est pas dans le vocabulaire.")


Mots similaires à 'roman_nom':
  bouquin_nom: 0.5913
  livre_nom: 0.5840
  feuilleton_nom: 0.5622
  polar_nom: 0.5528
  publier_ver: 0.5345

Mots similaires à 'poésie_nom':
  littérature_nom: 0.5188
  poème_nom: 0.5042
  poète_nom: 0.4918
  sonnet_nom: 0.4831
  poétique_adj: 0.4714

Mots similaires à 'auteur_nom':
  illustrateur_nom: 0.5687
  éditeur_nom: 0.5411
  roman_nom: 0.5259
  écrivain_nom: 0.5219
  texte_nom: 0.5211

Mots similaires à 'personnage_nom':
  individu_nom: 0.5343
  personnalité_nom: 0.5234
  acteur_nom: 0.4890
  inconnu_nom: 0.4744
  héros_nom: 0.4471

Mots similaires à 'métaphore_nom':
  formulation_nom: 0.4190
  heidegger_nam: 0.3999
  langage_nom: 0.3967
  locution_nom: 0.3967
  rhétorique_nom: 0.3932


In [ ]:
historical_terms = ["révolution_nom", "guerre_nom", "tuer_ver", "république_nom", "siècle_nom"]
for term in historical_terms:
    try:
        similar_words = model.wv.most_similar(term, topn=5)
        print(f"\nMots similaires à '{term}':")
        for word, similarity in similar_words:
            print(f"  {word}: {similarity:.4f}")
    except KeyError:
        print(f"Le mot '{term}' n'est pas dans le vocabulaire.")


Mots similaires à 'révolution_nom':
  guerre_nom: 0.5249
  révolutionnaire_nom: 0.5184
  révolutionnaire_adj: 0.5045
  insurrection_nom: 0.5005
  prolétariat_nom: 0.4660

Mots similaires à 'guerre_nom':
  bataille_nom: 0.5717
  révolution_nom: 0.5249
  combat_nom: 0.5125
  conflit_nom: 0.5002
  guerre_nam: 0.4633

Mots similaires à 'tuer_ver':
  assassiner_ver: 0.7172
  mourir_ver: 0.6211
  égorger_ver: 0.6039
  massacrer_ver: 0.5989
  suicider_ver: 0.5836

Mots similaires à 'république_nom':
  gouvernement_nom: 0.5829
  président_nom: 0.5585
  ministre_nom: 0.4974
  poincaré_nam: 0.4891
  procureur_nom: 0.4826

Mots similaires à 'siècle_nom':
  xixe_num: 0.6206
  millénaire_nom: 0.5647
  xxe_num: 0.5394
  xviie_num: 0.5328
  année_nom: 0.5322


In [ ]:
try:
    result = model.wv.most_similar(positive=['roi_nom', 'femme_nom'], negative=['homme_nom'], topn=1)
    print(f"roi - homme + femme = {result[0][0]}")
except KeyError:
    print("Un des mots n'est pas dans le vocabulaire.")

roi - homme + femme = reine_nom


In [ ]:
analogies = [
    ('auteur_nom', 'livre_nom', 'peintre_nom'),  # auteur est à livre ce que peintre est à ?
    ('paris_nam', 'france_nam', 'londres_nam'),  # Paris est à France ce que Londres est à ?
    ('passé_nom', 'présent_nom', 'avant_adv')     # passé est à présent ce que hier est à ?
]

In [ ]:
for a, b, c in analogies:
    try:
        result = model.wv.most_similar(positive=[b, c], negative=[a], topn=1)
        print(f"{a} est à {b} ce que {c} est à {result[0][0]}")
    except KeyError:
        print(f"Un des mots dans l'analogie ({a}, {b}, {c}) n'est pas dans le vocabulaire.")

auteur_nom est à livre_nom ce que peintre_nom est à tableau_nom
paris_nam est à france_nam ce que londres_nam est à angleterre_nam
passé_nom est à présent_nom ce que avant_adv est à maintenant_adv


In [ ]:
hugo_excerpt = """
c'était en effet un tableau digne d'intérêt que celui qui se déroulait
en ce moment sous le porche ogival de notre dame. la petite esmeralda était
assise sur le parvis, adossée contre un pilier. son bouc djali dormait à ses pieds.
"""

In [ ]:
word_pos_mapping = {
    "C'":"c_pron","en":"en_conj","effet":"effet_nom","un":"un_det","tableau": "tableau_nom","notre":"notre_adj",
    "digne":"digne_adj","d'":"d_prep","intérêt": "intérêt_nom", "que":"que_conj","celui":"celui_pron",
    "qui":"qui_pron","se":"se_pro","en":"en_prep","ce":"ce_det","moment": "moment_nom","sous":"sous_prep",
    "le":"le_det","porche": "porche_nom","ogival":"ogival_adj","la":"la_det","petite":"petite_adj", "dame": "dame_nom", "esmeralda": "esmeralda_nam",
    "était":"était_ver","adossée":"adossée_ver","contre":"contre_prep","un":"un_det","parvis": "parvis_nom", "pilier": "pilier_nom", "bouc": "bouc_nom",
    "son":"son_adj","à":"à_conj","ses":"ses_adj","bouc":"bouc_nom","dormait":"dormait_ver","djali": "djali_nam", "pieds": "pieds_nom", "fille": "fille_nom",
}

In [ ]:
words = hugo_excerpt.lower().split()
# Nettoyage très basique
words = [word.strip(".,;:!?()[]{}\"'") for word in words]
formatted_words = []
for word in words:
    if word in word_pos_mapping:
        formatted_words.append(word_pos_mapping[word])
    else:
        formatted_words.append(word)  # Garder le mot original si pas de mapping

# Filtrer les mots vides et ne garder que ceux présents dans le modèle
filtered_words = [word for word in formatted_words if len(word) > 3 and word in model.wv]

In [ ]:
print("Mots principaux de l'extrait avec leurs étiquettes grammaticales:", filtered_words)

# Trouver le "centre sémantique" de l'extrait
if filtered_words:
    excerpt_vector = np.mean([model.wv[word] for word in filtered_words], axis=0)
    most_similar = model.wv.most_similar(positive=[excerpt_vector], topn=10)
    print("\nThèmes principaux détectés dans l'extrait:")
    for word, similarity in most_similar:
        print(f"  {word}: {similarity:.4f}")

Mots principaux de l'extrait avec leurs étiquettes grammaticales: ['effet_nom', 'tableau_nom', 'digne_adj', 'moment_nom', 'porche_nom', 'ogival_adj', 'dame_nom', 'parvis_nom', 'pilier_nom', 'son_adj', 'bouc_nom', 'ses_adj', 'pieds_nom']

Thèmes principaux détectés dans l'extrait:
  porche_nom: 0.5719
  ogival_adj: 0.5515
  pilier_nom: 0.5458
  parvis_nom: 0.5434
  pilastre_nom: 0.4999
  notre: 0.4823
  fronton_nom: 0.4783
  victoires_nam: 0.4777
  église_nom: 0.4773
  voûte_nom: 0.4766


## Testez avec vos propres fichiers txt

In [ ]:
!pip install stanza

In [ ]:
import os
import stanza
from gensim.models import Word2Vec

In [ ]:
stanza.download('fr')
nlp = stanza.Pipeline('fr', processors='tokenize,pos,lemma')

In [ ]:
folder_path = "/content/"

define_batch_size = 1024  # Taille dynamique du batch

def batch_process_texts(folder_path, batch_size=define_batch_size):
    all_sentences = []
    texts = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())

    paragraphs = "\n".join(texts).split("\n")
    batches = [paragraphs[i:i + batch_size] for i in range(0, len(paragraphs), batch_size)]

    for batch in batches:
        batch_text = "\n".join(batch)
        doc = nlp(batch_text)
        for sentence in doc.sentences:
            lemmatized_sentence = [word.lemma for word in sentence.words if word.lemma is not None]
            all_sentences.append(lemmatized_sentence)

    return all_sentences

sentences = batch_process_texts(folder_path, batch_size=define_batch_size)

In [ ]:
model = Word2Vec(sentences, vector_size=300, window=20, min_count=2, sg=1)
model.save("word2vec_model.model")

def word_similarity(word1, word2):
    return model.wv.similarity(word1, word2)

def word_analogy(word1, word2, word3):
    return model.wv.most_similar(positive=[word2, word3], negative=[word1])


def most_similar_words(word, topn=10):
    try:
        similar_words = model.wv.most_similar(word, topn=topn)
        for w, sim in similar_words:
            print(f"{w}: {sim:.4f}")
    except KeyError:
        print(f"Le mot '{word}' n'est pas dans le vocabulaire du modèle.")

In [ ]:
with open("vectors.tsv", "w", encoding="utf-8") as vec_file, open("metadata.tsv", "w", encoding="utf-8") as meta_file:
    for word in model.wv.index_to_key:
        vector = model.wv[word]
        vec_file.write("\t".join([str(x) for x in vector]) + "\n")
        meta_file.write(word + "\n")

Vous pouvez vous servir des derniers fichiers générés dans l'interface en ligne [tensorflow](https://projector.tensorflow.org/)